In [ ]:
import pandas as pd
import numpy as np
import toughio

# --- USER CONFIGURATION ---
input_csv = "Block Model base.csv"
output_mesh = "MESH"
# --------------------------

print(f"--- Reading {input_csv} ---")

# 1. Load Data
try:
    # Read CSV, skipping Leapfrog metadata lines starting with #
    df = pd.read_csv(input_csv, comment='#')
    df.columns = df.columns.str.strip() # Clean headers
except Exception as e:
    print(f"Error reading CSV: {e}")
    exit(1)

# 2. Analyze Geometry (Auto-detect Grid)
print("Analyzing grid geometry...")

# Get unique, sorted coordinates
x_unique = np.sort(df['X'].unique())
y_unique = np.sort(df['Y'].unique())
z_unique = np.sort(df['Z'].unique())

# Calculate block counts
nx = len(x_unique)
ny = len(y_unique)
nz = len(z_unique)

# Calculate grid spacing (dx, dy, dz)
# We assume constant spacing (Regular Block Model)
dx_val = x_unique[1] - x_unique[0]
dy_val = y_unique[1] - y_unique[0]
dz_val = z_unique[1] - z_unique[0]

# Create arrays of spacing for toughio (e.g., [10.0, 10.0, ...])
dx = np.full(nx, dx_val)
dy = np.full(ny, dy_val)
dz = np.full(nz, dz_val)

# Determine Origin (Bottom-South-West corner)
# Note: Leapfrog centroids are center-points. TOUGH origin is the corner.
origin = [
    x_unique[0] - dx_val/2,
    y_unique[0] - dy_val/2,
    z_unique[0] - dz_val/2
]

print(f"Grid Detected: {nx} x {ny} x {nz} blocks")
print(f"Spacing: dx={dx_val:.1f}, dy={dy_val:.1f}, dz={dz_val:.1f}")
print(f"Origin: {origin}")

# 3. Create Mesh using meshmaker
print("Generating structured TOUGH mesh...")
mesh = toughio.meshmaker.structured_grid(
    dx=dx, 
    dy=dy, 
    dz=dz, 
    origin=origin
)

# 4. Map Materials
# CRITICAL: We must ensure the CSV data order matches the Mesh block order.
# toughio/pyvista structured grids usually order by X (fastest), then Y, then Z (slowest).
# We sort the dataframe to match this ordering before extracting materials.
print("Mapping materials...")
df_sorted = df.sort_values(by=['Z', 'Y', 'X'], ascending=[True, True, True])

# Extract the material column (last column)
material_col = df.columns[-1]
materials = df_sorted[material_col].astype(str).to_numpy()

# Assign to mesh
mesh.add_cell_data("Material", materials)

# 5. Write Output
print(f"Writing {output_mesh}...")
toughio.write_mesh(output_mesh, mesh)

print("------------------------------------------------")
print(f"Success! '{output_mesh}' created.")
print("Copy this file to your TOUGHREACT folder.")

--- Reading Block Model base.csv ---
Analyzing grid geometry...
Grid Detected: 79 x 58 x 4 blocks
Spacing: dx=1000.0, dy=1000.0, dz=1000.0
Origin: [np.float64(286108.2957), np.float64(5068232.802452875), np.float64(-2100.0)]
Generating structured TOUGH mesh...
Mapping materials...
Writing MESH...
------------------------------------------------
Success! 'MESH' created.
Copy this file to your TOUGHREACT folder.
